# 改一下原数据集的格式

In [1]:
#import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')
import pandas as pd

In [ ]:
#now_data = 'aline'
#'data/' + now_data +

app_launch_log = pd.read_csv( '../../Desktop/datasets/app_launch_log.txt',sep='\t',header=None)
app_launch_log = app_launch_log.sort_index(by=[0,1])
app_launch_log = app_launch_log.rename({0:'user_id',1:'launch_day'})
app_launch_log.to_csv('app_launch_log.csv',index=False)

user_activity_log = pd.read_csv('../../Desktop/datasets/user_activity_log.txt',sep='\t',header=None)
user_activity_log = user_activity_log.sort_index(by=[0,1])
user_activity_log = user_activity_log.rename({0:'user_id',1:'activity_day',2:'page',3:'video_id',4:'author_id',5:'action_type'})
user_activity_log.to_csv('user_activity_log.csv',index=False)

user_register_log = pd.read_csv( '../../Desktop/datasets/user_register_log.txt',sep='\t',header=None)
user_register_log = user_register_log.sort_index(by=[0,1])
user_register_log = user_register_log.rename({0:'user_id',1:'register_day',2:'register_type',3:'device type'})
user_register_log.to_csv('user_register_log.csv',index=False)

video_create_log = pd.read_csv( '../../Desktop/datasets/video_create_log.txt',sep='\t',header=None)
video_create_log = video_create_log.sort_index(by=[0,1])
video_create_log = video_create_log.rename({0:'user_id',1:'create_day'})
video_create_log.to_csv('video_create_log.csv',index=False)

C:\Users\87613\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  """
C:\Users\87613\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  # Remove the CWD from sys.path while we load stuff.


# 划分数据集部分

In [3]:
import pandas as pd
import numpy as np

In [4]:
all_dataSet_path = 'all_dataSet.csv'
one_dataSet_train_path = 'one_dataSet_train_'
one_dataSet_test_path = 'one_dataSet_test_'
two_dataSet_train_path = 'two_dataSet_train_'
two_dataSet_test_path = 'two_dataSet_test_'
three_dataSet_train_path = 'three_dataSet_train_'

launch = pd.read_csv('app_launch_log.csv')
activity = pd.read_csv('user_activity_log.csv')
register = pd.read_csv('user_register_log.csv')
video = pd.read_csv('video_create_log.csv')

def cut_data_as_time(dataSet_path, new_dataSet_path, begin_day, end_day):
    temp_register = register[(register['register_day'] >= begin_day) & (register['register_day'] <= end_day)]
    temp_create = video[(video['create_day'] >= begin_day) & (video['create_day'] <= end_day)]
    temp_launch = launch[(launch['launch_day'] >= begin_day) & (launch['launch_day'] <= end_day)]
    temp_activity = activity[(activity['activity_day'] >= begin_day) & (activity['activity_day'] <= end_day)]
    
    temp_register.to_csv(new_dataSet_path + 'register.csv',index=False)
    temp_create.to_csv(new_dataSet_path + 'create.csv',index=False)
    temp_launch.to_csv(new_dataSet_path + 'launch.csv',index=False)
    temp_activity.to_csv(new_dataSet_path + 'activity.csv',index=False)

    print(u'根据起始和结束时间将数据集取出完成，存入：',new_dataSet_path)

def generate_dataSet():
    print("开始划分数据集......")
    # 以1-16数据 预测17-23某用户是否活跃
    begin_day = 1
    end_day = 16
    cut_data_as_time(all_dataSet_path,one_dataSet_train_path,begin_day,end_day)
    begin_day = 17
    end_day = 23
    cut_data_as_time(all_dataSet_path,one_dataSet_test_path,begin_day,end_day)
    print("第一数据集（包括数据集和结果集）划分完成......")
    # 以8-23数据 预测24-30某用户是否活跃
    begin_day = 8
    end_day = 23
    cut_data_as_time(all_dataSet_path,two_dataSet_train_path,begin_day,end_day)
    begin_day = 24
    end_day = 30
    cut_data_as_time(all_dataSet_path,two_dataSet_test_path,begin_day,end_day)
    print("第二数据集（包括数据集和结果集）划分完成......")
    # 以15-30数据 预测31-37某用户是否活跃
    begin_day = 15
    end_day = 30
    cut_data_as_time(all_dataSet_path,three_dataSet_train_path,begin_day,end_day)
    print("第三数据集（包括数据集和结果集）划分完成......")
    print("成功划分三个数据集......")
    
generate_dataSet()

开始划分数据集......
根据起始和结束时间将数据集取出完成，存入： one_dataSet_train_
根据起始和结束时间将数据集取出完成，存入： one_dataSet_test_
第一数据集（包括数据集和结果集）划分完成......
根据起始和结束时间将数据集取出完成，存入： two_dataSet_train_
根据起始和结束时间将数据集取出完成，存入： two_dataSet_test_
第二数据集（包括数据集和结果集）划分完成......
根据起始和结束时间将数据集取出完成，存入： three_dataSet_train_
第三数据集（包括数据集和结果集）划分完成......
成功划分三个数据集......


# 生成特征

In [ ]:
from collections import Counter
from scipy.stats import stats
import pandas as pd
import numpy as np

In [ ]:
one_dataSet_train_path = 'one_dataSet_train_'
one_dataSet_test_path = 'one_dataSet_test_'
two_dataSet_train_path = 'two_dataSet_train_'
two_dataSet_test_path = 'two_dataSet_test_'
three_dataSet_train_path = 'three_dataSet_train_'

train_path = 'train.csv'
test_path = 'test.csv'

register = 'register.csv'
create = 'create.csv'
launch = 'launch.csv'
activity = 'activity.csv'

# 构建训练集与测试集与特征
# 获取所有id 查看对应id是否在测试集里面出现过
def get_train_label(train_path,test_path):
    train_reg = pd.read_csv(train_path + register,usecols = ['user_id'])
    train_cre = pd.read_csv(train_path + create,usecols = ['user_id'])
    train_lau = pd.read_csv(train_path + launch,usecols = ['user_id'])
    train_act = pd.read_csv(train_path + activity,usecols = ['user_id'])
    train_data_id = np.unique(pd.concat([train_reg,train_cre,train_lau,train_act]))
    
    test_reg = pd.read_csv(test_path + register,usecols = ['user_id'])
    test_cre = pd.read_csv(test_path + create,usecols = ['user_id'])
    test_lau = pd.read_csv(test_path + launch,usecols = ['user_id'])
    test_act = pd.read_csv(test_path + activity,usecols = ['user_id'])
    test_data_id = np.unique(pd.concat([test_reg,test_cre,test_lau,test_act]))
    
    train_label = []
    for i in train_data_id:
        for i in test_data_id:
            train_label.append(1)
        else:
            train_label.append(0)
    train_data = pd.DataFrame()
    train_data['user_id'] = train_data_id
    train_data['label'] = train_label
    return train_data

def get_test(test_path):
    test_reg = pd.read_csv(test_path + register,usecols = ['user_id'])
    test_cre = pd.read_csv(test_path + create,usecols = ['user_id'])
    test_lau = pd.read_csv(test_path + launch,usecols = ['user_id'])
    test_act = pd.read_csv(test_path + activity,usecols = ['user_id'])
    test_data_id = np.unique(pd.concat([test_reg,test_cre,test_lau,test_act]))
    test_data = pd.DataFrame()
    test_data['user_id'] = test_data_id
    return test_data

def get_create_feature(row):
    feature = pd.Series()
    feature['user_id'] = list(row['user_id'])[0]
    return feature

def get_register_feature(row):
    feature = pd.Series()
    feature['user_id'] = list(row['user_id'])[0]
    return feature

def get_launch_feature(row):
    feature = pd.Series()
    feature['user_id'] = list(row['user_id'])[0]
    return feature

def get_activity_feature(row):
    feature = pd.Series()
    feature['user_id'] = list(row['user_id'])[0]
    return feature

def deal_feature(path,user_id):
    reg = pd.read_csv(path + register)
    cre = pd.read_csv(path + create)
    lau = pd.read_csv(path + launch)
    act = pd.read_csv(path + activity)
    feature = pd.DataFrame()
    feature['user_id'] = user_id
    
    # w...
    cre['max_day'] = np.max(reg['register_day'])
    cre_feature = cre.groupby('user_id',sort=Ture).apply(get_create_feature)
    feature = pd.merge(feature,pd.DataFrame(cre_feature),on='user_id',how='left')
    print("create表特征提取完毕")
    
    # w...
    reg['max_day'] = np.max(reg['register_day'])
    reg_feature = reg.groupby('user_id',sort=Ture).apply(get_register_feature)
    feature = pd.merge(feature,pd.DataFrame(reg_feature),on='user_id',how='left')
    print("register表特征提取完毕")
    
    # w...
    lau['max_day'] = np.max(reg['register_day'])
    lau_feature = lau.groupby('user_id',sort=Ture).apply(get_launch_feature)
    feature = pd.merge(feature,pd.DataFrame(lau_feature),on='user_id',how='left')
    print("launch表特征提取完毕")
    
    # w......
    act['max_day'] = np.max(reg['register_day'])
    act_feature = act.groupby('user_id',sort=Ture).apply(get_activity_feature)
    feature = pd.merge(feature,pd.DataFrame(act_feature),on='user_id',how='left')
    print("activity表特征提取完毕")
    return feature

# 构建特征
def get_data_feature():
    one_train_data = get_train_label(one_dataSet_train_path,one_dataSet_test_path)
    one_feature = deal_feature(one_dataSet_train_path,one_train_data['user_id'])
    one_feature['label'] = one_train_data['label']
    print("第一组训练数据特征提取完毕")
    
    two_train_data = get_train_label(two_dataSet_train_path,two_dataSet_test_path)
    two_feature = deal_feature(two_dataSet_train_path,two_train_data['user_id'])
    two_feature['label'] = two_train_data['label']
    print("第二组训练数据特征提取完毕")
    
    train_feature = pd.concat([one_feature,two_feature])
    train_feature.to_csv(train_path,index=False)
    print("训练数据存储完毕")
    
    test_data = get_test(three_dataSet_train_path)
    test_feature = deal_feature(three_dataSet_train_path,test_data['user_id'])
    test_feature.to_csv(test_path,index=False)
    print("测试数据存储完毕")

get_data_feature()

# 模型预测（lgb）

In [ ]:
import pandas as pd
from sklearn import model_selection
import numpy as np
import sklearn.metrics


print('开始处理特征......')

train_path = 'train_and_test/train.csv'
test_path = 'train_and_test/test.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

used_feature = ['create_count', 'create_day_diff_mean', 'create_day_diff_std', 'create_day_diff_max',
                'create_day_diff_min', 'create_mode', 'last_day_cut_max_day',
                'register_type', 'device_type', 'register_day_cut_max_day',
                'launch_count', 'launch_day_diff_mean', 'launch_day_diff_std',
                'launch_day_diff_max', 'launch_day_diff_min', 'launch_day_diff_kur',
                'launch_day_diff_ske', 'launch_day_diff_last', 'launch_day_cut_max_day',
                'activity_count',
                'activity_day_diff_mean',
                'activity_day_diff_std',
                'activity_day_diff_max', 'activity_day_diff_min', 'activity_day_diff_kur',
                'activity_day_diff_ske',
                'activity_day_diff_last',
                '0_page_count', '1_page_count', '2_page_count', '3_page_count', '4_page_count',
                '0_page_count_div_sum', '1_page_count_div_sum', '2_page_count_div_sum',
                '3_page_count_div_sum', '4_page_count_div_sum',
                '0_action_count',
                '1_action_count', '2_action_count', '3_action_count', '4_action_count',
                '5_action_count', '0_action_count_div_sum', '1_action_count_div_sum',
                '2_action_count_div_sum', '3_action_count_div_sum',
                '4_action_count_div_sum', '5_action_count_div_sum',
                'video_id_mode', 'author_id_mode', 'activity_count_mean',
                'activity_count_std', 'activity_count_max', 'activity_count_min',
                'activity_count_kur', 'activity_count_ske', 'activity_count_last',
                'activity_diff_count_mean', 'activity_diff_count_std', 'activity_diff_count_max',
                'activity_diff_count_min', 'activity_diff_count_kur', 'activity_diff_count_ske',
                'activity_diff_count_last', 'activity_page0_mean', 'activity_page0_std',
                'activity_page0_max', 'activity_page0_min', 'activity_page0_kur', 'activity_page0_ske',
                'activity_page0_last', 'activity_page1_mean', 'activity_page1_std', 'activity_page1_max',
                'activity_page1_min', 'activity_page1_kur', 'activity_page1_ske', 'activity_page1_last',
                'activity_page2_mean', 'activity_page2_std', 'activity_page2_max', 'activity_page2_min',
                'activity_page2_kur', 'activity_page2_ske', 'activity_page2_last', 'activity_page3_mean',
                'activity_page3_std', 'activity_page3_max', 'activity_page3_min', 'activity_page3_kur',
                'activity_page3_ske', 'activity_page3_last', 'activity_page4_mean', 'activity_page4_std',
                'activity_page4_max', 'activity_page4_min', 'activity_page4_kur', 'activity_page4_ske',
                'activity_page4_last', 'activity_type0_mean', 'activity_type0_std', 'activity_type0_max',
                'activity_type0_min', 'activity_type0_kur', 'activity_type0_ske', 'activity_type0_last',
                'activity_type1_mean', 'activity_type1_std', 'activity_type1_max', 'activity_type1_min',
                'activity_type1_kur', 'activity_type1_ske', 'activity_type1_last', 'activity_type2_mean',
                'activity_type2_std', 'activity_type2_max', 'activity_type2_min', 'activity_type2_kur',
                'activity_type2_ske', 'activity_type2_last', 'activity_type3_mean', 'activity_type3_std',
                'activity_type3_max', 'activity_type3_min', 'activity_type3_kur', 'activity_type3_ske',
                'activity_type3_last', 'activity_type4_mean', 'activity_type4_std', 'activity_type4_max',
                'activity_type4_min', 'activity_type4_kur', 'activity_type4_ske', 'activity_type4_last',
                'activity_type5_mean', 'activity_type5_std', 'activity_type5_max', 'activity_type5_min',
                'activity_type5_kur', 'activity_type5_ske', 'activity_type5_last', 'activity_day_cut_max_day',
                'max_activity_day',
                 'create_sub_register', 'activity_sub_register', 'launch_sub_register',
                ]
used_feature = np.array(used_feature)
print(used_feature)
importance_feature = [21,60,54,71,106,44,50,27,33,58,43,11,19,35,64,32,45,9,37,143,142,10,7,18,8]
used_feature = used_feature[np.array(importance_feature)]
print(used_feature)
train_feature = train[used_feature]
test_feature = test[used_feature]
label = train['label']

# 切分训练
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(train_feature, label, test_size=0.2,random_state=1017)
# train_feature = X_train
# label = Y_train

print('特征处理完毕......')


###################### lgb ##########################
import lightgbm as lgb

print('载入数据......')
lgb_train = lgb.Dataset(train_feature, label)
lgb_eval = lgb.Dataset(X_test, Y_test, reference=lgb_train)


print('开始训练......')
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc', 'binary_logloss'}
}
# params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'regression',
#     'metric': {'l2'}
# }

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=58,
                valid_sets=lgb_eval
                )
gbm.save_model('model/lgb_model.txt')

temp = gbm.predict(X_test)
temp[temp>0.5]=1
temp[temp<0.5]=0
print('结果：' + str(sklearn.metrics.f1_score(Y_test, temp)))
print('特征重要性：'+ str(list(gbm.feature_importance())))


########################## 保存结果 ############################
pre = gbm.predict(test_feature)
df_result = pd.DataFrame()
df_result['user_id'] = test['user_id']
df_result['result'] = pre
df_result.to_csv('result/lgb_result.csv', index=False)
pre[pre >= 0.5]=1
pre[pre < 0.5]=0
pre = map(int,pre)
print('为1的个数：' + str(len(np.where(np.array(pre)==1)[0])))
print('为0的个数：' + str(len(np.where(np.array(pre)==0)[0])))

In [ ]:
import pandas as pd
from sklearn import model_selection
import numpy as np
import sklearn.metrics

lgb = pd.read_csv('result/lgb_result.csv')
print(lgb)
res = lgb[lgb['result'] >= 0.5]
print(len(res))
del res['result']
res.to_csv('result/dealed_result.txt', index=False, header=False)